<span style="font-size: 30px;">Taller Machine learning I</span>

<span style="font-size: 20px;">Integrantes</span>

* Nombre: Maria del mar Ipia Guzmán
* Cédula: 1214726595
* Correo: maria.ipia@udea.edu.co

  

- Nombre: Johan Arbey Vargas Mejía
- Cédula: 1152195956
- Correo: johan.vargas@udea.edu.co

# Librerias

In [20]:
import json
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import re
import zipfile

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Carga de datos

In [2]:
path = os.path.join('..', 'data', 'youtube_model.parquet')
youtube_model = pd.read_parquet(path)

In [3]:
youtube_model.head(2)

,views,likes,dislikes,comment_count,days,title_length,tag_count,category
0,2418783,97185,6146,12703,0,62,4,Entertainment
1,343168,10172,666,2146,0,32,27,Entertainment


In [4]:
youtube_regresion = pd.get_dummies(youtube_model, columns=['category'], dtype=int)

In [5]:
x = youtube_regresion.drop('likes', axis=1)
y = youtube_regresion['likes']

# Escalamiento de datos

In [6]:
min_max = MinMaxScaler()

numeric_scale = min_max.fit_transform(x)

numeric_scale = pd.DataFrame(numeric_scale, columns=x.columns)

In [7]:
numeric_scale

,views,dislikes,comment_count,days,title_length,tag_count,category_Comedy,category_Entertainment,category_Film & Animation,category_Howto & Style,category_Music,category_News & Politics,category_Other,category_People & Blogs,category_Science & Technology,category_Sports
0,0.010738,0.003671,0.009330,0.000237,0.546296,0.044118,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.001521,0.000398,0.001576,0.000237,0.268519,0.382353,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.009303,0.001188,0.012866,0.000474,0.194444,0.191176,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.003629,0.000465,0.002521,0.000474,0.296296,0.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.003665,0.000071,0.000250,0.000237,0.574074,0.397059,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34664,0.104355,0.009551,0.038508,0.006406,0.314815,0.338235,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34665,0.036670,0.002420,0.045983,0.006406,0.453704,0.382353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
34666,0.007482,0.000827,0.001951,0.006406,0.231481,0.191176,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
34667,0.004733,0.000616,0.002932,0.006406,0.750000,0.338235,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Regresión lineal

In [8]:
x_train, x_test, y_train, y_test = train_test_split(
    numeric_scale, y, train_size=0.8, random_state=1234, shuffle=True
)

In [9]:
lr = LinearRegression()

In [10]:
lr.fit(x_train, y_train)

LinearRegression()

In [11]:
y_predict = lr.predict(x_test)

In [12]:
r2_score(y_test, y_predict)

0.9165351684739675

# Random Forest Regressor

In [17]:
rfr = RandomForestRegressor(
            n_estimators = 10,
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,   #out-of-bag samples
            n_jobs       = -1,
            random_state = 123)

In [18]:
rfr.fit(x_train, y_train)

C:\Users\mdmgu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


RandomForestRegressor(max_features='auto', n_estimators=10, n_jobs=-1,
                      random_state=123)

In [19]:
y_predictrf = rfr.predict(x_test)

In [22]:
rmse = mean_squared_error(y_test, y_predictrf, squared=False)
print(f"El error (rmse) de test es: {rmse}")

r2 = r2_score(y_test, y_predictrf)
print(f"El error (r2) de test es: {r2}")

El error (rmse) de test es: 27730.08194034284
El error (r2) de test es: 0.9854130144686074


* Usamos GridSearchCV para buscar el mejor modelo

In [23]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': [5, 7, 9],
    'max_depth': [None, 5, 10]}

In [24]:
rf = RandomForestRegressor(random_state=1234)

In [29]:
grid_search = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid, 
    cv=5, 
    scoring='r2')

In [30]:
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=1234),
             param_grid={'max_depth': [None, 5, 10], 'max_features': [5, 7, 9],
                         'n_estimators': [50, 100, 200]},
             scoring='r2')

In [32]:
params_dt = grid_search.best_params_
r_model = grid_search.best_estimator_

In [33]:
y_pred = r_model.predict(x_test)

In [34]:
accuracy = r2_score(y_test, y_pred)
print("Exactitud del modelo: {:.2f}%".format(accuracy * 100))

Exactitud del modelo: 99.34%


In [35]:
params_dt

{'max_depth': None, 'max_features': 7, 'n_estimators': 200}